## Working with Data
PyTorch has two primitives to work with data: `torch.utils.data.DataLoader` and `torch.utils.data.Dataset`. Dataset stores the samples and their corresponding `labels`, and `DataLoader` wraps an iterable around the Dataset.

In [8]:
import torch
from torch import nn
from torch.utils.data import DataLoader
from torchvision import datasets
from torchvision.transforms import ToTensor


print(torch.__version__)

2.3.0+cpu


PyTorch offers domain-specific libraries such as TorchText, TorchVision, and TorchAudio, all of which include datasets. For this tutorial, we will be using a TorchVision dataset.

The torchvision.datasets module contains Dataset objects for many real-world vision data like CIFAR, COCO (full list here). In this tutorial, we use the FashionMNIST dataset. Every TorchVision Dataset includes two arguments: transform and target_transform to modify the samples and labels respectively.

In [6]:
training_data = datasets.FashionMNIST(
    root="data",
    train=True,
    download = True,
    transform= ToTensor(),

)

test_data = datasets.FashionMNIST(
    root="data",
    train=False,
    download=True,
    transform=ToTensor()
)

We pass the Dataset as an argument to DataLoader. This wraps an iterable over our dataset, and supports automatic batching, sampling, shuffling and multiprocess data loading. Here we define a batch size of 64, i.e. each element in the dataloader iterable will return a batch of 64 features and labels.

In [10]:
batch =  64


#Create Dataloader
train_dataloader = DataLoader(training_data, batch_size=batch)
test_dataloader = DataLoader(test_data, batch_size=batch)


for X, y in test_dataloader:
  print(f"Shape of X [N, C, H, W]:{X.shape}")
  print(f"Shape of y {y.shape}")
  break





Shape of X [N, C, H, W]:torch.Size([64, 1, 28, 28])
Shape of y torch.Size([64])


## Creating Models

To define a neural network in PyTorch, we create a class that inherits from nn.Module. We define the layers of the network in the `__init__` function and specify how data will pass through the network in the forward function. To accelerate operations in the neural network, we move it to the GPU or MPS if available.

In [12]:
#Get gpu or cpu or mps for training
device = (
    "cuda"
    if torch.cuda.is_available()
    else "mps"
    if torch.backends.mps.is_available()
    else "cpu"
)

print(f"Using {device} device")

#Defining a Model
class NeuralNetwork(nn.Module):
  def __init__(self):
    super().__init__()
    self.flatten = nn.Flatten()
    self.linear_relu_stack = nn.Sequential(
        nn.Linear(28*28, 512),
        nn.ReLU(),
        nn.Linear(512,512),
        nn.ReLU(),
        nn.Linear(512,10)
    )
  def forward(self,x):
    x = self.flatten(x)
    logits = self.linear_relu_stack(x)
    return logits

model = NeuralNetwork().to(device)
print(model)


Using cpu device
NeuralNetwork(
  (flatten): Flatten(start_dim=1, end_dim=-1)
  (linear_relu_stack): Sequential(
    (0): Linear(in_features=784, out_features=512, bias=True)
    (1): ReLU()
    (2): Linear(in_features=512, out_features=512, bias=True)
    (3): ReLU()
    (4): Linear(in_features=512, out_features=10, bias=True)
  )
)


## Optimizing the Model Parameters
To train a model, we need a loss function and an optimizer.

In [13]:
loss_fn = nn.CrossEntropyLoss()
optimizer = torch.optim.SGD(model.parameters(),lr=1e-3)

In a single training loop, the model makes predictions on the training dataset (fed to it in batches), and backpropagates the prediction error to adjust the model’s parameters.

In [14]:
def train(dataloader, model,loss_fn, optimzer):
  size = len(dataloader.dataset)
  model.train()
  for batch , (X, y) in enumerate(dataloader):
    X, y = X.to(device), y.to(device)

    #Compute prediction error
    pred = model(X)
    loss = loss_fn(pred, y)

    #backpropogation
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    if batch %100 == 0:
      loss, current = loss.item(), (batch+1)*len(X)
      print(f"loss:{loss:7f} [{current:5d}/{size:>5d}]")

In [15]:
def test(dataloader, model, loss_fn):
    size = len(dataloader.dataset)
    num_batches = len(dataloader)
    model.eval()
    test_loss, correct = 0, 0
    with torch.no_grad():
        for X, y in dataloader:
            X, y = X.to(device), y.to(device)
            pred = model(X)
            test_loss += loss_fn(pred, y).item()
            correct += (pred.argmax(1) == y).type(torch.float).sum().item()
    test_loss /= num_batches
    correct /= size
    print(f"Test Error: \n Accuracy: {(100*correct):>0.1f}%, Avg loss: {test_loss:>8f} \n")

The training process is conducted over several iterations (epochs). During each epoch, the model learns parameters to make better predictions. We print the model’s accuracy and loss at each epoch; we’d like to see the accuracy increase and the loss decrease with every epoch.

In [17]:
epochs = 5

for t in range(epochs):
  print(f"Epoch {t+1}\n------------------------")
  train(train_dataloader, model,loss_fn, optimizer)
  test(test_dataloader, model, loss_fn)

print("Done!")

Epoch 1
------------------------
loss:2.304532 [   64/60000]
loss:2.289857 [ 6464/60000]
loss:2.269620 [12864/60000]
loss:2.265147 [19264/60000]
loss:2.238503 [25664/60000]
loss:2.215836 [32064/60000]
loss:2.222673 [38464/60000]
loss:2.188067 [44864/60000]
loss:2.180319 [51264/60000]
loss:2.143541 [57664/60000]
Test Error: 
 Accuracy: 42.8%, Avg loss: 2.141984 

Epoch 2
------------------------
loss:2.156656 [   64/60000]
loss:2.143682 [ 6464/60000]
loss:2.087671 [12864/60000]
loss:2.104527 [19264/60000]
loss:2.038887 [25664/60000]
loss:1.985150 [32064/60000]
loss:2.010131 [38464/60000]
loss:1.924928 [44864/60000]
loss:1.926612 [51264/60000]
loss:1.855635 [57664/60000]
Test Error: 
 Accuracy: 49.8%, Avg loss: 1.855386 

Epoch 3
------------------------
loss:1.889915 [   64/60000]
loss:1.858770 [ 6464/60000]
loss:1.742885 [12864/60000]
loss:1.789586 [19264/60000]
loss:1.664247 [25664/60000]
loss:1.622648 [32064/60000]
loss:1.645928 [38464/60000]
loss:1.539350 [44864/60000]
loss:1.565821

## Saving Models
A common way to save a model is to serialize the internal state dictionary (containing the model parameters).

In [18]:
torch.save(model.state_dict(),"model.pth")
print("Saved Pytorch Model State")

Saved Pytorch Model State


## Loading Models
The process for loading a model includes re-creating the model structure and loading the state dictionary into it.

In [19]:
model = NeuralNetwork().to(device)
model.load_state_dict(torch.load("model.pth"))

<All keys matched successfully>

In [21]:
classes = [
    "T-shirt/top",
    "Trouser",
    "Pullover",
    "Dress",
    "Coat",
    "Sandal",
    "Shirt",
    "Sneaker",
    "Bag",
    "Ankle boot",
]

model.eval()
x, y = test_data[0][0], test_data[0][1]
with torch.no_grad():
    x = x.to(device)
    pred = model(x)
    predicted, actual = classes[pred[0].argmax(0)], classes[y]
    print(f'Predicted: "{predicted}", Actual: "{actual}"')

Predicted: "Ankle boot", Actual: "Ankle boot"
